In [1]:
from __future__ import division  

import numpy as np
import pandas as pd
from geopy.distance import vincenty
import json

In [2]:
# read the dataset, using json.loads to read the data structure --- list.

taxi_trips = pd.read_csv('/home/rui/Documents/MLProject/data/modified-crab/every-trip-data.csv')
trip_trace = taxi_trips['trip_trace']
new_triptrace = []
for i in xrange(trip_trace.shape[0]):
    new_triptrace.append(json.loads(trip_trace.iloc[i]))

taxi_trips['trip_trace'] = new_triptrace

In [3]:
taxi_trips.shape

(453048, 4)

In [4]:
trip_trace = []

In [5]:
sampling_rate_bias = 10

In [6]:
for row_index, row in taxi_trips.iterrows():
        
    single_trip = row[3]
    trip_length = len(single_trip)
    
    valid_single_trip = []
    
    all_periods = {} # store the difference between each pair of time points in a trajectory
    
    for i in range(0, trip_length):
        for h in range(i+1, trip_length):
            all_periods[(i, h)] = (abs(single_trip[h][1] - single_trip[i][1] - 60))
    
    valid_trip = []
    valid_trip.append(0) # put the start location in it.
    for i in range(0, trip_length-1):
        if not i in valid_trip:
            continue
        
        sample_list = [] # list record the sample rate with abs - 60
        smallest_abs = 999999
        closest_id = -1
        
        for h in range(i+1, trip_length):
            if all_periods[(i, h)] <  smallest_abs:
                smallest_abs = all_periods[(i, h)]
                closest_id = h
        
        if smallest_abs <= sampling_rate_bias:
            valid_trip.append(closest_id)
        else:
            break
    
    for valid_trip_id in valid_trip:
        valid_single_trip.append(single_trip[valid_trip_id])
    
    trip_trace.append(valid_single_trip)

In [7]:
len(trip_trace)

453048

In [8]:
taxi_trips['trip_trace'] = trip_trace
taxi_trips.head()

,taxi_id,time_stamp,week_day,trip_trace
0,enyenewl,1.212620e+09,2,"[[[37.78401, -122.40783], 1212618405.0], [[37...."
1,enyenewl,1.212620e+09,2,"[[[37.79243, -122.42614], 1212619843.0], [[37...."
2,enyenewl,1.212625e+09,3,"[[[37.78684, -122.3998], 1212621717.0], [[37.7..."
3,enyenewl,1.212626e+09,3,"[[[37.8085, -122.41004], 1212625046.0], [[37.8..."
4,enyenewl,1.212626e+09,3,"[[[37.7934, -122.39326], 1212625685.0], [[37.7..."


In [9]:
taxi_trips.to_csv('/home/rui/Documents/MLProject/data/modified-crab/after-resample.csv', index=None)